<a href="https://colab.research.google.com/github/AnovaYoung/IoT_PhysicalTherapy/blob/main/TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile
import os
import pandas as pd
import numpy as np



zip_file_path = '/Users/anovayoungers/Downloads/physical+therapy+exercises+dataset.zip'

extraction_directory = '/Users/anovayoungers/Downloads/physical_therapy_dataset'

with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_directory)

    print(f'Files Extracted to {extraction_directory}')

for root, dirs, files in os.walk(extraction_directory, topdown=True):
    for name in files:
        print(os.path.join(root, name))
    for name in  dirs:
        print(os.path.join(root, name))


def load_data(file_path):
    """Load data from a specified file path into a pandas DataFrame."""
    try :
        data = pd.read_csv(file_path, sep="\t", header=None)
        return data
    except Exception as e:
        print(f"Error Loading Data: {e} ")
        return None

file_path = '/Users/anovayoungers/Downloads/physical_therapy_dataset/s1/e1/u1/test.txt'
data = load_data(file_path)

def load_data_from_directory(directory_path):
    sensor_data = []
    session_metadata = []
    times_metadata = []

    for root, dirs, files in os.walk(directory_path, topdown=True):
        parts = root.split(os.sep)
        subject = next((part for part in parts if part.startswith('s')), None)
        exercise = next((part for part in parts if part.startswith('e')), None)
        sensor_unit = next((part for part in parts if part.startswith('u')), None)

        for name in files:
            file_path = os.path.join(root, name)
            if name == 'test.txt':
                data = pd.read_csv(file_path, sep=";", header=0)
                data['subject'] = subject
                data['exercise'] = exercise
                data['sensor_unit'] = sensor_unit
                sensor_data.append(data)
            elif name == 'template_session.txt':
                session_data = pd.read_csv(file_path, sep=";", header=0)
                session_data['subject'] = subject
                session_data['exercise'] = exercise
                session_data['sensor_unit'] = sensor_unit
                session_metadata.append(session_data)
            elif name == 'template_times.txt':
                times_data = pd.read_csv(file_path, sep=";", header=0)
                times_data['subject'] = subject
                times_data['exercise'] = exercise

                times_metadata.append(times_data)

    sensor_df = pd.concat(sensor_data, ignore_index=True)
    session_df = pd.concat(session_metadata, ignore_index=True)
    times_df = pd.concat(times_metadata, ignore_index=True)

    return sensor_df, session_df, times_df

extraction_directory = '/Users/anovayoungers/Downloads/physical_therapy_dataset'
sensor_df, session_df, times_df = load_data_from_directory(extraction_directory)

In [ ]:
print(times_df.head())

   execution type  start    end subject exercise
0               1  168.0  267.0      s5       e5
1               2  519.0  588.0      s5       e5
2               3  805.0  890.0      s5       e5
3               1  175.0  315.0      s5       e2
4               2  595.0  671.0      s5       e2


In [ ]:
print(sensor_df.head(100))

    time index     acc_x     acc_y     acc_z     gyr_x     gyr_y     gyr_z  \
0            1  1.141452  9.587617  1.343498 -0.070101  0.005029 -0.011399   
1            2  1.164059  9.602035  1.463472 -0.034009  0.010527 -0.011461   
2            3  1.164136  9.639337  1.478518  0.002012 -0.015333 -0.011723   
3            4  1.194045  9.661973  1.374475  0.020135 -0.000033 -0.011679   
4            5  1.178997  9.647259  1.329349  0.025554  0.001804 -0.010769   
..         ...       ...       ...       ...       ...       ...       ...   
95          96  1.296696  9.564875 -0.910514  0.014767 -0.008208  0.023501   
96          97  1.206768  9.482702 -0.792701  0.084830 -0.012439 -0.021883   
97          98  1.222116  9.939922 -1.239535  0.156035  0.006158 -0.002965   
98          99  1.266282  9.491561 -1.269886  0.005385  0.003686 -0.008040   
99         100  1.131626  9.536581 -1.145015  0.013635 -0.015405 -0.024414   

       mag_x     mag_y     mag_z subject exercise sensor_unit  

In [ ]:
print(times_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   execution type  120 non-null    int64  
 1   start           120 non-null    float64
 2   end             120 non-null    float64
 3   subject         120 non-null    object 
 4   exercise        120 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 4.8+ KB
None


In [ ]:
print(sensor_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1101390 entries, 0 to 1101389
Data columns (total 13 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   time index   1101390 non-null  int64  
 1   acc_x        1101390 non-null  float64
 2   acc_y        1101390 non-null  float64
 3   acc_z        1101390 non-null  float64
 4   gyr_x        1101390 non-null  float64
 5   gyr_y        1101390 non-null  float64
 6   gyr_z        1101390 non-null  float64
 7   mag_x        1101390 non-null  float64
 8   mag_y        1101390 non-null  float64
 9   mag_z        1101390 non-null  float64
 10  subject      1101390 non-null  object 
 11  exercise     1101390 non-null  object 
 12  sensor_unit  1101390 non-null  object 
dtypes: float64(9), int64(1), object(3)
memory usage: 109.2+ MB
None


In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np

data = sensor_df[['acc_x', 'acc_y', 'acc_z']].values # Input data
target = sensor_df['acc_x'].values  # Target variable

n_input = 10
generator = TimeseriesGenerator(data, target, length=n_input, batch_size=1)

for i in range(len(generator)):
    x, y = generator[i]
    print(f'Input sequence: {x}, Target value: {y}')
    if i >= 2:  # Stops after 2 batches
        break


Input sequence: [[[1.141452 9.587617 1.343498]
  [1.164059 9.602035 1.463472]
  [1.164136 9.639337 1.478518]
  [1.194045 9.661973 1.374475]
  [1.178997 9.647259 1.329349]
  [1.163999 9.662428 1.284303]
  [1.194028 9.692118 1.299835]
  [1.17904  9.684691 1.307034]
  [1.164054 9.647173 1.373928]
  [1.156551 9.632215 1.388705]]], Target value: [1.164066]
Input sequence: [[[1.164059 9.602035 1.463472]
  [1.164136 9.639337 1.478518]
  [1.194045 9.661973 1.374475]
  [1.178997 9.647259 1.329349]
  [1.163999 9.662428 1.284303]
  [1.194028 9.692118 1.299835]
  [1.17904  9.684691 1.307034]
  [1.164054 9.647173 1.373928]
  [1.156551 9.632215 1.388705]
  [1.164066 9.662167 1.359025]]], Target value: [1.179049]
Input sequence: [[[1.164136 9.639337 1.478518]
  [1.194045 9.661973 1.374475]
  [1.178997 9.647259 1.329349]
  [1.163999 9.662428 1.284303]
  [1.194028 9.692118 1.299835]
  [1.17904  9.684691 1.307034]
  [1.164054 9.647173 1.373928]
  [1.156551 9.632215 1.388705]
  [1.164066 9.662167 1.35902

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

X_train, X_temp, y_train, y_temp = train_test_split(data, target, test_size=0.2, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

train_generator = TimeseriesGenerator(X_train, y_train, length=n_input, batch_size=1)
val_generator = TimeseriesGenerator(X_val, y_val, length=n_input, batch_size=1)

model = Sequential([
    LSTM(50, activation='relu', input_shape=(n_input, 3)),  # the 3 features: acc_x, acc_y, acc_z
    Dense(1)  # This means predict the next value of acc_x
])
model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(generator, epochs=10, batch_size=256, verbose=1, callbacks=[early_stopping], validation_data=val_generator)


Epoch 1/10
1101380/1101380 [==============================] - 1237s 1ms/step - loss: 1.5211 - val_loss: 54.4043
Epoch 2/10
1101380/1101380 [==============================] - 1238s 1ms/step - loss: 29.6963 - val_loss: 52.1510
Epoch 3/10
1101380/1101380 [==============================] - 1233s 1ms/step - loss: 0.2780 - val_loss: 54.5494
Epoch 4/10
1101380/1101380 [==============================] - 1495s 1ms/step - loss: 10.3007 - val_loss: 52.7331
Epoch 5/10
1101380/1101380 [==============================] - 1571s 1ms/step - loss: 0.1849 - val_loss: 49.1595
Epoch 6/10
1101380/1101380 [==============================] - 1590s 1ms/step - loss: 0.2763 - val_loss: 50.5445
Epoch 7/10
1101380/1101380 [==============================] - 1587s 1ms/step - loss: 0.2467 - val_loss: 50.8342
Epoch 8/10
1101380/1101380 [==============================] - 1865s 2ms/step - loss: 0.2998 - val_loss: 52.3259


In [ ]:
predictions = model.predict(val_generator)


110129/110129 [==============================] - 49s 447us/step


In [ ]:
print(len(y_val), len(predictions))


110139 110129


In [ ]:
# Truncate y_val to match the length of predictions
y_val_aligned = y_val[:len(predictions)]

print(len(y_val_aligned), len(predictions))

110129 110129


In [ ]:
actual_vs_predicted = pd.DataFrame({
    'Actual': y_val_aligned,
    'Predicted': predictions.flatten()
})

actual_vs_predicted['Index'] = range(len(actual_vs_predicted))

actual_vs_predicted.to_csv('/Users/anovayoungers/Downloads/actual_vs_predicted.csv', index=False)
